# Find all must support dependent profiles in StructureDefinitions

prerequisite - need a completed local build

- see FHIR-39723
- for hard coded path search all SD for all reference with must support targets
  - including all references with a single target
  - exclude add'l USCDI
  - ignore choice of targets for now 
- associated url with title
- save as simple csv in IG data file
- use liquid template to geenerate table in IG

TODO ... for add'l USCDI most of these are choice of target

  - adjust the boolean flags "addl_uscdi" and "ms" as follows:
  
  |table type|addl_uscdi flag|ms flag|
  |----|----|----|
  |*Must Support* targets for *Must Support* Reference type elements|False|True|
  |*Must Support* targets for *Add'l USCDI* Reference type elements|True|True|
  |*Choice of at least one* target for *Must Support* Reference type elements|False|False|
  |*Choice of at least one* target for *Add'l USCDI*  Reference type elements|True|False|

### Imports and Constants

In [1]:
from pathlib import Path
from json import loads, dumps
import pandas as pd
snapshot_path =  Path('/Users/ehaas/Documents/FHIR/US-Core/output')  # use this  NEED a completed local build
my_path = snapshot_path
ig_base_url = 'http://hl7.org/fhir/us/core/'  # US Core don't forget the trailing /
fhir_base = 'http://hl7.org/fhir/' # canonical url for FHIR don't forget the trailing /
out_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/input/data')

#===============  adjust the boolean flags "addl_uscdi" and "ms"  ==========
# addl_uscdi=True
addl_uscdi=True
# ms = False
ms = False
#===========================================================================

### Function to get the References

- aglorithm:
  - all refs with single target = ms references
  - all refs with >1 target look at ms extension if true = ms reference
    - if all false choice of reference
  - if >1 target look and no ms extension = choice of reference
  - if other then error - fix profile

In [2]:
def get_my_dict(addl_uscdi=False, ms = True):
  print(f'addl_uscdi = {addl_uscdi}')
  print(f'ms = {ms}')
  my_dict = {}
  derived_map={}

  for i in sorted(my_path.glob('Struct*.json')):
    obj = loads(i.read_text()) #dict
    if  obj['type'] == 'Extension':
        continue
    # print(f"US Core Profile = {obj['title']}")
    # derived_map[obj['url']] = obj['baseDefinition']
    my_urls = set()
    for e in obj['snapshot']['element']:
        
      try:
        if addl_uscdi:
          if e["extension"] and 'http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement' in [x['url'] for x in e["extension"]]:
            pass 
        elif e["mustSupport"]:
            # print(f'for {e["id"]}  e["mustSupport"] = {e["mustSupport"]}')
            pass  
        else:
           continue             
      except KeyError as e:
          # print(f"======== no {e} in element !========")
          pass
      else:
          # print(e['id'])
          try:
            ref_type = [type for type in e['type'] if type['code'] == 'Reference'][0]
            if ref_type:
              # print(f"ADDL USCDI Reference type elements: {e['id']}\n ref_type = {ref_type}")  # == all USCDI Reference type ==
              pass
          except IndexError:  
              pass
          except KeyError:  # no Type
              pass
          # ms_targets
          else:
            # if only one choice
            try:
              if len(ref_type['targetProfile']) == 1: # single reference is MS
                  # print(">>>>>>>>>>> ms_target_type")  # == all USCDI Reference type ==
                  if ms:
                      my_urls.update(ref_type['targetProfile'])
              else:  # >1` references
                try:
                  if ref_type['_targetProfile']:
                    # print(">>>>>>>>>>>potential ms_target_types")  # == all potential USCDI Reference type ==
                    pass
                except KeyError as e:
                  # print(f"<<<<<<<<<<<<< choice_target_type")  # == all USCDI Reference type ==
                  if not ms:
                      my_urls.update(ref_type['targetProfile'])
                  # pass  
                else:
                  ms_targets = [ms_target for i,ms_target in enumerate(ref_type['targetProfile']) if ref_type['_targetProfile'][i]['extension'][0]['valueBoolean']]
                  if ms_targets:  # some '_targetProfile'  = true
                    # print(f"=============== ms_targets = {ms_targets}")  # == ms targets ==
                    if ms:
                      my_urls.update(ms_targets)
                  else:  # all '_targetProfile'  = false
                    # print(f"=============== ms_targets = {ms_targets}")  # == no ms targets ==
                    if not ms:
                      my_urls.update(ref_type['targetProfile'])
            except: # no targetprofile for reference type = e.g for any reference types.
              pass
    my_dict[obj['title']] = list(my_urls)

  # #- iterate through and combine lists of profiles from parent profiles.   
  # for k,v in derived_map.items():
  #     if v in my_dict.keys():
  #         # print(k,v,my_dict[k],my_dict[v])
  #         my_dict[k] = my_dict[k] + my_dict[v]

  print(f'======For {"Must Support Targets" if ms else "Choice"} Target for {"ADDL USCDI" if addl_uscdi else "Must Support"} Elements==============')
  print(f"my_dict = {my_dict}") 
  print('==========================================================================================================================================')
  return my_dict

### Map the urls to titles and create CSV for use in IG as liquid template

- get the ig resource to map to the profile titles
- create df

In [3]:
ig_path = my_path / 'ImplementationGuide-hl7.fhir.us.core.json'
canonical = 'http://hl7.org/fhir/us/core/'
# print(type(ig_path))
# print(ig_path.read_text())
ig_obj = loads(ig_path.read_text()) #dict

def title_map(url):
  try:
    title = [resource['name'] for resource in ig_obj['definition']['resource'] if resource['reference']['reference'] == url.split(canonical)[-1]][0]
  except IndexError:
    #  print(f"no title for {url} use {url.split('/')[-1]}")
     title = url.split('/')[-1]
  return title 

def get_new_dict(my_dict): # map urls to Titles for rendering in IG
  new_dict = {}
  for (k,v) in my_dict.items():
      targets = [title_map(target) for target in v if target != 'http://hl7.org/fhir/StructureDefinition/Resource']  # remove generic resource target
      new_dict[k] = sorted(targets)
  return new_dict
    

my_dict = get_my_dict(addl_uscdi=addl_uscdi, ms=ms)  #dictionary of references for each profile
if addl_uscdi == True and ms == True:  # compare to *Must Support* targets for *Must Support* Reference type elements
  ms_dict = get_my_dict(addl_uscdi=False, ms=True)
  for k,v in ms_dict.items():
     my_dict[k] = [x for x in my_dict[k] if x not in k] # whatever is not in *Must Support* targets for *Must Support* Reference type elements
# print(f"my_dict={my_dict}")
new_dict = get_new_dict(my_dict)
# print(f"new_dict = {new_dict}")

df = pd.DataFrame.from_dict(new_dict, orient='index' )  # transposes rows to columns and assigns the index to the profile row
df

addl_uscdi = True
ms = False
======For Choice Target for ADDL USCDI Elements==============
my_dict = {'US Core Pediatric Head Occipital Frontal Circumference Percentile Profile': [], 'US Core Pediatric BMI for Age Observation Profile': [], 'US Core Pediatric Weight for Height Observation Profile': [], 'US Core ADI DocumentReference Profile': [], 'US Core AllergyIntolerance Profile': [], 'US Core Average Blood Pressure Profile': [], 'US Core Blood Pressure Profile': [], 'US Core BMI Profile': [], 'US Core Body Height Profile': [], 'US Core Body Temperature Profile': [], 'US Core Body Weight Profile': [], 'US Core Care Experience Preference Profile': [], 'US Core CarePlan Profile': [], 'US Core CareTeam Profile': [], 'US Core Condition Encounter Diagnosis Profile': [], 'US Core Condition Problems and Health Concerns Profile': [], 'US Core Coverage Profile': [], 'US Core DiagnosticReport Profile for Laboratory Results Reporting': [], 'US Core DiagnosticReport Profile for Report and Note E

,0,1,2,3,4,5,6
US Core Pediatric Head Occipital Frontal Circumference Percentile Profile,None,None,None,None,None,None,None
US Core Pediatric BMI for Age Observation Profile,None,None,None,None,None,None,None
US Core Pediatric Weight for Height Observation Profile,None,None,None,None,None,None,None
US Core ADI DocumentReference Profile,None,None,None,None,None,None,None
US Core AllergyIntolerance Profile,None,None,None,None,None,None,None
US Core Average Blood Pressure Profile,None,None,None,None,None,None,None
US Core Blood Pressure Profile,None,None,None,None,None,None,None
US Core BMI Profile,None,None,None,None,None,None,None
US Core Body Height Profile,None,None,None,None,None,None,None
US Core Body Temperature Profile,None,None,None,None,None,None,None


### Save as csv and update headers
  - this is a hack - save df as csv, retrieve csv and resave with headers.

In [4]:
file_name = f'{"addl_uscdi_" if addl_uscdi else "ms_"}{"refs" if ms else "choice_refs"}.csv'
print(file_name)
file_path = out_path / file_name
df.to_csv(file_path)
df = pd.read_csv(file_path)
targets = [f'target{i+1}' for i in range(len(df.columns)-1)]
print(targets)
headers = ['profile'] + targets
df.to_csv(file_path, header=headers,)

addl_uscdi_choice_refs.csv
['target1', 'target2', 'target3', 'target4', 'target5', 'target6', 'target7']
